In [1]:
import os
import rasterio
import geopandas as gpd
from shapely.geometry import box
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from rasterio import features

from rasterio.features import rasterize
from rasterio.transform import from_origin
import pandas as pd
from rasterio import features
from shapely.geometry import Polygon, MultiPolygon

# FUNCTION 

The raster_to_polygons_optimized function is designed to convert a raster dataset into a GeoDataFrame of polygons, where each polygon represents an individual cell from the raster. 

The function takes a single argument, raster_path, which specifies the file path of the raster to be processed. It begins by opening the raster file using rasterio, extracting the transformation parameters, reading the first band of data, and identifying the no-data value specified in the raster's metadata. 

The function then identifies all valid cells—those not equal to the no-data value—and iterates over these cells. For each valid cell, it calculates the polygon geometry that corresponds to the cell's spatial extent using the raster's transformation parameters and assigns the cell's value to this polygon. 

These polygons and their associated values are then aggregated into a GeoDataFrame, which is assigned the same Coordinate Reference System (CRS) as the source raster. 

The resulting GeoDataFrame, containing a 'value' column for cell values and a 'geometry' column for the corresponding polygons, is returned by the function. This optimized approach ensures efficient processing by focusing only on valid raster cells, thereby excluding areas with no-data values and reducing computational overhead.

In [2]:
def raster_to_polygons_optimized(raster_path):
    """
    Convert a raster dataset to a GeoDataFrame where each valid cell (non-no-data) is represented as a polygon.

    This function opens a raster file, reads its first band, and converts each valid cell into a polygon. The value
    of each cell is retained and associated with the corresponding polygon. The resulting GeoDataFrame contains
    polygons with their associated values and is set to the same CRS as the input raster.

    Parameters:
    - raster_path (str): The file path of the raster to be processed.

    Returns:
    - gpd.GeoDataFrame: A GeoDataFrame with two columns: 'value' containing the cell values and 'geometry'
      containing the corresponding polygon geometries for each valid raster cell.
    """
    # Extract the base filename without extension to use as the column name
    filename = os.path.splitext(os.path.basename(raster_path))[0]

    # Open the raster file using rasterio
    with rasterio.open(raster_path) as src:
        
        # Extract the affine transform for the raster
        transform = src.transform
        # Read the first band of the raster
        data = src.read(1)
        # Retrieve the no-data value set in the raster's metadata
        nodata = src.nodata

    # Identify valid cells (cells not equal to the no-data value)
    valid_cells = np.argwhere(data != nodata)

    polygons = []  # List to store polygon geometries
    values = []  # List to store the values of the valid cells

    # Iterate over the indices of valid cells to create polygons
    for (j, i) in valid_cells:
        # Extract the value of the current cell
        value = data[j, i]
        # Create a polygon geometry for the current cell based on its coordinates and the raster's transform
        polygon = box(
            transform[2] + i * transform[0],  # Minx
            transform[5] + (j + 1) * transform[4],  # Miny
            transform[2] + (i + 1) * transform[0],  # Maxx
            transform[5] + j * transform[4]  # Maxy
        )
        # Append the current polygon and its value to their respective lists
        polygons.append(polygon)
        values.append(value)

    # Create a GeoDataFrame with the polygons and their associated values
    gdf = gpd.GeoDataFrame({filename: values, 'geometry': polygons})
    # Set the CRS of the GeoDataFrame to match the CRS of the source raster
    gdf.crs = src.crs

    return gdf


In [3]:
#gdf_test = raster_to_polygons_optimized()

A raster transform is a set of coefficients that define the spatial relationship between the coordinates of pixels in a raster dataset and the geographic coordinates on the surface of the Earth. This transform allows you to map the row and column indices of pixels in the raster grid to their corresponding geographic locations (such as latitude and longitude or coordinates in a projected coordinate system).

The raster transform typically includes six coefficients, often represented in an affine transformation matrix. These coefficients are:

a: The width of a pixel in the units of the coordinate system (e.g., meters).
b and d: These coefficients typically rotate the raster, but in most north-up images, these are zero.
c: The X-coordinate of the upper-left corner of the upper-left pixel.
e: The height of a pixel in the units of the coordinate system, which is usually a negative value because pixel row indices increase downward, while geographic coordinates usually increase upward.
f: The Y-coordinate of the upper-left corner of the upper-left pixel.
In the context of the rasterio library, the transform is often used to calculate the geographic coordinates of the center of a pixel given its row and column indices, or vice versa. The transform object can perform these calculations through methods like * (to convert pixel coordinates to geographic coordinates) and ~ (to convert geographic coordinates to pixel coordinates).

An example affine transform could look like this: [a, b, c, d, e, f], where a, d, and e are the pixel sizes and rotation terms, and c and f give the geographic coordinates of the upper-left pixel.

Understanding the raster transform is crucial for tasks like georeferencing, where you need to align the raster data with a specific geographic location, or when performing spatial analyses that require the conversion of pixel coordinates to real-world geographic locations.

# Process Raster(s) and Create 'INPUT REPORTING UNITS'

In [4]:

# Folder containing all your raster files
raster_folder = r"C:\Users\bsf31\Documents\post-meds\data\signal\riparian_restoration"
# Get a list of all raster files in the folder (assuming they are TIFF files)
raster_files = [f for f in os.listdir(raster_folder) if f.endswith('.tif')]

# Initialize variables to store the smallest resolution
smallest_res = None

# Iterate through each raster file to find the smallest resolution
for raster_file in raster_files:
    with rasterio.open(os.path.join(raster_folder, raster_file)) as src:
        # src.res returns a tuple (width, height) of pixels
        if smallest_res is None or (src.res[0] < smallest_res[0] and src.res[1] < smallest_res[1]):
            smallest_res = src.res

# smallest_res now contains the smallest resolution by width and height
print("Smallest resolution:", smallest_res)

# Initialize an empty GeoDataFrame to store the combined results
combined_gdf = gpd.GeoDataFrame()

# Process each raster file
for raster_file in raster_files:
    raster_path = os.path.join(raster_folder, raster_file)
    with rasterio.open(raster_path) as src:
        # Check if the raster's resolution matches the most common resolution
        if src.res != smallest_res:
            print(f"Skipping {raster_file} due to mismatching resolution.")
            continue  # Skip this file

    # If resolution matches, process the raster
    current_gdf = raster_to_polygons_optimized(raster_path)
    
    # Merge the current GeoDataFrame with the combined one
    if combined_gdf.empty:
        combined_gdf = current_gdf
    else:
        combined_gdf = combined_gdf.merge(current_gdf, on='geometry', how='outer')

# Save the combined GeoDataFrame to a GeoPackage
output_gpkg_path = r'C:\Users\bsf31\Documents\post-meds\data\signal\input_reporting_units\input_reporting_units.gpkg'
combined_gdf.to_file(output_gpkg_path, layer='WHRHabitatType', driver='GPKG')


Smallest resolution: (196.0, 196.0)


In [15]:
output_gpkg_path

'C:\\Users\\bsf31\\Documents\\post-meds\\data\\signal\\input_reporting_units\\input_reporting_units.gpkg'

# Optional - Append New Single File Raster Data

In [ ]:
# Path to the existing GeoPackage and the new raster file
existing_gpkg_path = output_gpkg_path
# Load the existing GeoPackage into a GeoDataFrame
existing_gdf = gpd.read_file(existing_gpkg_path, layer='combined_layer')



In [ ]:
new_raster_path = 'path_to_new_raster_file.tif'
# Process the new raster to a GeoDataFrame
new_gdf = raster_to_polygons_optimized(new_raster_path)

# Merge the new GeoDataFrame with the existing one
# Ensure the merge is done based on geometry or another suitable common attribute
updated_gdf = existing_gdf.merge(new_gdf, on='geometry', how='outer')

# Update the GeoPackage with the updated GeoDataFrame
# This example overwrites the existing layer; you could also choose to add a new layer
updated_gdf.to_file(existing_gpkg_path, layer='combined_layer', driver='GPKG')


In [7]:
study_path = r"C:\Users\bsf31\Documents\post-meds\data\signal\RWMP_AREA_2229.gpkg"
study_area = gpd.read_file(study_path)

# REFERENCE RASTER

In [10]:
output_crs = 'EPSG:2229'
output_resolution = (33.75336306985997936, -34.37544019661349637)


In [9]:
study_area_bounds =study_area.total_bounds
study_area_bounds

array([5807047.11341745, 1961361.28632066, 6127434.03567656,
       2040906.05493562])

# Vector to Raster

In [11]:
def vector_to_raster(input_vector, output_raster, attribute, study_area_bounds, value_mapping, single_value=None, resolution=(abs(0.00026949458523585647), abs(-0.00026949458523585647)), dtype='uint16'):
    # Check if input_vector is a GeoDataFrame or a file path
    if isinstance(input_vector, gpd.GeoDataFrame):
        gdf = input_vector
    else:
        # Read the input vector file (GeoPackage or Shapefile) into a GeoDataFrame
        gdf = gpd.read_file(input_vector)
    # Reproject the GeoDataFrame to the desired CRS (EPSG:2229)
    gdf = gdf.to_crs(epsg=2229)

    # Ensure that categorical column is string
    gdf[attribute] = gdf[attribute].astype(str)
   

   # If single_value is None, convert the attribute column to numerical values
    # If single_value is provided, set the attribute column to the provided single_value
    if single_value is None:
        gdf[attribute] = gdf[attribute].replace(value_mapping).astype(dtype)
    else:
        gdf[attribute] = single_value


    # Use the study area bounds to define the dimensions and transform of the output raster
    minx, miny, maxx, maxy = study_area_bounds
    width = 9492
    height = 2314
    out_transform = rasterio.transform.from_bounds(minx, miny, maxx, maxy, width, height)


    # Define the metadata for the output raster file
    out_meta = {
        'driver': 'GTiff',
        'width': width,
        'height': height,
        'count': 1,
        'dtype': dtype,
        'crs': 'EPSG:2229',
        'transform': out_transform,
        'nodata': 0
    }
    
    # Open the output raster file for writing with the specified metadata
    with rasterio.open(output_raster, 'w', **out_meta) as dst:
        # Create a generator of tuples containing the geometry and attribute value for each feature in the input vector data
        shapes = ((geom, value) for geom, value in zip(gdf['geometry'], gdf[attribute]))
        
        # Burn the geometries and their corresponding attribute values into a raster array
        burned = features.rasterize(
            shapes=shapes,         # The generator of geometry-attribute tuples
            fill=0,                # The default value for pixels not covered by any geometry
            out_shape=(height, width), # The shape of the output raster array (number of rows and columns)
            transform=out_transform,   # The affine transformation matrix that maps pixel coordinates to the coordinate reference system
            dtype=dtype            # The data type of the raster array
        )
        
        # Write the burned raster array to the output raster file
        dst.write(burned, 1)

# Functions to call Vector to Raster

In [12]:
# To write text file of value mapping
def write_value_mapping(value_mapping, output_file):
    with open(output_file, 'w') as f:
        for key, value in value_mapping.items():
            f.write(f'{key}: {value}\n')

In [13]:
def process_columns(input_vector, output_dir, study_area_bounds, resolution, columns, value_mapping, single_value=None, file_name=None):
    if file_name is None:
        file_name = os.path.splitext(os.path.basename(input_vector))[0]

    for column in columns:
        column_output_dir = os.path.join(output_dir, column)
        os.makedirs(column_output_dir, exist_ok=True)

        output_raster = f"{column_output_dir}/{file_name}_{column}_raster.tif"
        vector_to_raster(input_vector, output_raster, column, study_area_bounds, value_mapping[column], single_value=single_value, resolution=resolution)
        

In [14]:
def process_files_from_list(file_list, output_dir, study_area_bounds, resolution, columns, single_value=None):
    # Generate a global value mapping from the list of unique values for each column
    global_value_mapping = {column: set() for column in columns}
    
    for file_path in file_list:
        gdf = gpd.read_file(file_path)
        for column in columns:
            global_value_mapping[column].update(gdf[column].astype(str).unique())
    
    for column in columns:
        global_value_mapping[column] = {value: idx for idx, value in enumerate(sorted(list(global_value_mapping[column])), 1)}
        print(f"Global value mapping for {column}:")
        for value in global_value_mapping[column]:
            print(f"{value}: {global_value_mapping[column][value]}")
    
    # Write the global value mapping for each column to separate .txt files in the corresponding column folder
    for column in columns:
        column_output_dir = os.path.join(output_dir, column)
        os.makedirs(column_output_dir, exist_ok=True)

        output_value_mapping_file = f"{column_output_dir}/{column}_global_value_mapping.txt"
        write_value_mapping(global_value_mapping[column], output_value_mapping_file)

    # Process each file with the global value mapping
    for file_path in file_list:
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        process_columns(file_path, output_dir, study_area_bounds, columns=columns, resolution=resolution, value_mapping=global_value_mapping, single_value=single_value, file_name=file_name)

In [15]:
# Where files will save, can add subfolders if desired
output_dir = os.path.join(r"C:\Users\bsf31\Documents\post-meds\data\signal\test_vector")
os.makedirs(output_dir, exist_ok=True)
test_vector = [os.path.join(output_dir, 'calveg_cwhr_rwmp.gpkg')]

In [16]:
process_files_from_list(test_vector, output_dir, study_area_bounds, output_resolution, columns=['WHRHabitatType'])


Global value mapping for WHRHabitatType:
Annual Grassland: 1
Barren: 2
Blue Oak Woodland: 3
Blue Oak-Foothill Pine: 4
Chamise-Redshank Chaparral: 5
Closed-Cone Pine-Cypress: 6
Coastal Oak Woodland: 7
Coastal Scrub: 8
Cropland: 9
Deciduous Orchard: 10
Desert Wash: 11
Eucalyptus: 12
Evergreen Orchard: 13
Freshwater Emergent Wetland: 14
Lacustrine: 15
Mixed Chaparral: 16
Montane Hardwood: 17
Montane Hardwood-Conifer: 18
Montane Riparian: 19
Pasture: 20
Perennial Grassland: 21
Riverine: 22
Saline Emergent Wetland: 23
Sierran Mixed Conifer: 24
Urban: 25
Valley Oak Woodland: 26
Valley-Foothill Riparian: 27
Vineyard: 28
Water: 29


# Append Vector Data

In [46]:
new_raster_path = r"C:\Users\bsf31\Documents\post-meds\data\signal\test_vector\WHRHabitatType\calveg_cwhr_rwmp_WHRHabitatType_raster.tif"
# Process the new raster to a GeoDataFrame
new_gdf = raster_to_polygons_optimized(new_raster_path)

# Merge the new GeoDataFrame with the existing one
# Ensure the merge is done based on geometry or another suitable common attribute
updated_gdf = existing_gdf.merge(new_gdf, on='geometry', how='outer')

# Update the GeoPackage with the updated GeoDataFrame
# This example overwrites the existing layer; you could also choose to add a new layer
updated_gdf.to_file(existing_gpkg_path, layer='new_combined_layer', driver='GPKG')


In [ ]:
def append_vector_to_gpkg(vector_path, gpkg_path, input_layer, output_layer, dissolve=False, aggfunc=None):
    """
    Appends data from a vector file to an existing layer in a GeoPackage and optionally aggregates the result,
    saving it as a new layer within the same GeoPackage.

    Parameters:
    - vector_path (str): The file path of the vector file to be appended.
    - gpkg_path (str): The file path of the GeoPackage to which the data will be appended.
    - input_layer (str): The name of the existing layer in the GeoPackage to which the vector data will be appended.
    - output_layer (str): The name of the new layer in the GeoPackage where the result will be saved.
    - dissolve (bool, optional): Whether to dissolve (aggregate) the spatial join results. Defaults to False.
    - aggfunc (str or dict, optional): The aggregation function to use when dissolving. This parameter is only used if dissolve is True. Defaults to None.
    """

    # Load the existing GeoPackage layer
    gdf_units = gpd.read_file(gpkg_path, layer=input_layer)

    # Load the vector file
    gdf_vector = gpd.read_file(vector_path)

    # Ensure the CRS of the vector data matches that of the GeoPackage layer
    if gdf_units.crs != gdf_vector.crs:
        gdf_vector = gdf_vector.to_crs(gdf_units.crs)

    # Perform a spatial join between the GeoPackage layer and the vector data
    gdf_joined = gpd.sjoin(gdf_units, gdf_vector, how="left", op="intersects")

    # Check if dissolving (aggregation) is required
    if dissolve:
        # If no specific aggregation function is provided, raise an error
        if aggfunc is None:
            raise ValueError("An aggregation function must be provided when dissolve is True.")
        
        # Dissolve (aggregate) the joined data
        gdf_result = gdf_joined.dissolve(by="index_left", aggfunc=aggfunc)
    else:
        # If dissolving is not required, the result is the spatially joined data
        gdf_result = gdf_joined

    # Save the result as a new layer in the GeoPackage
    gdf_result.to_file(gpkg_path, layer=output_layer, driver='GPKG')


 GeoPandas, when you use the `dissolve` method, the `aggfunc` parameter specifies how to aggregate data for columns other than the one used for grouping (in this case, the one specified by the by parameter). The `aggfunc` can be a function, a string, or a dictionary mapping column names to operations. Common aggregation functions include:

* '`sum`': Sum of values.
* '`mean`': Mean of values.
* '`max`': Maximum value.
* '`min`': Minimum value.
* '`median`': Median of values.
* '`std`': Standard deviation of values.
* '`var`': Variance of values.
* '`first`': First value.
* '`last`': Last value.
* '`count`': Count of non-null values.

You can also use any function that is accepted by the aggregate method of pandas DataFrame. This includes functions from the numpy library like numpy.sum or numpy.mean, and you can also define your own custom aggregation functions.

If you need to apply different aggregation functions to different columns, you can pass a dictionary to aggfunc, where keys are column names and values are functions or names of functions. For example:

`aggfunc={'population': 'sum', 'area': 'mean'}`

This would sum the values in the 'population' column and calculate the mean of the values in the 'area' column for each group.

It's important to note that the aggregation function(s) you choose should make sense for the type of data you're working with and the specific analysis or outcome you're aiming for. For instance, summing up categorical data or taking the mean of ordinal data might not yield meaningful results.